### Instance analysis
In this notebook, we just want to analyze the instances of the dataset

In [1]:
import os

import pandas as pd
from tqdm import tqdm

from trigger_arc_tsp.gurobi_tsp_model import GurobiTSPModel
from trigger_arc_tsp.instance import Instance
from trigger_arc_tsp.utils import INSTANCES_DIR

In [6]:
# list instances in the instances directory
instance_names = (os.listdir(os.path.join(INSTANCES_DIR, "instances_release_2")))

# sort according to the instance number. This is grf{num}.txt
instance_names.sort(key=lambda x: int(x[3:-4]))

# load all instances
instances = []
for instance_name in tqdm(instance_names):
    instance = Instance.load_instance_from_file(os.path.join(INSTANCES_DIR, "instances_release_2", instance_name))
    instances.append(instance)

  0%|          | 0/34 [00:00<?, ?it/s]

reading instance file
initialize Instance object
Instance loading: Loading indices from cache


 21%|██        | 7/34 [00:00<00:01, 20.58it/s]

reading instance file
initialize Instance object
Instance loading: Loading indices from cache
reading instance file
initialize Instance object
Instance loading: Loading indices from cache
reading instance file
initialize Instance object
Instance loading: Loading indices from cache
reading instance file
initialize Instance object
Instance loading: Loading indices from cache
reading instance file
initialize Instance object
Instance loading: Loading indices from cache
reading instance file
initialize Instance object
Instance loading: Loading indices from cache
reading instance file
initialize Instance object
Instance loading: Loading indices from cache
reading instance file
initialize Instance object
Instance loading: Loading indices from cache
reading instance file
initialize Instance object
Instance loading: Loading indices from cache
reading instance file
initialize Instance object
Instance loading: Loading indices from cache
reading instance file


 35%|███▌      | 12/34 [00:00<00:00, 27.76it/s]

...

 94%|█████████▍| 32/34 [00:04<00:00,  4.73it/s]

reading instance file
initialize Instance object
Instance loading: Loading indices from cache


 97%|█████████▋| 33/34 [00:04<00:00,  3.79it/s]

reading instance file
initialize Instance object
Instance loading: Loading indices from cache


100%|██████████| 34/34 [00:05<00:00,  6.35it/s]


In [5]:
# Print the N, A, and R values for each instance
res = pd.DataFrame({
    "name": [instance.name.split("/")[1] for instance in instances],
    "N": [instance.N for instance in instances],
    "A": [instance.A for instance in instances],
    "R": [instance.R for instance in instances]
})

print(res, "\n")

# Counts of N
print(res["N"].value_counts())



          name    N     A       R
0   grf101.txt   18    90    1144
1   grf102.txt   22   132    2135
2   grf103.txt   22   110    1824
3   grf104.txt   27   162    3410
4   grf105.txt   26   130    2664
5   grf106.txt   32   192    4985
6   grf107.txt   37   222    6860
7   grf108.txt   37   222    6860
8   grf109.txt   37   222    6860
9   grf110.txt   37   222    6860
10  grf111.txt   42   252    9035
11  grf112.txt   50   350   15168
12  grf113.txt   58   464   23583
13  grf114.txt   66   594   34640
14  grf115.txt   74   740   48699
15  grf116.txt   82   902   66120
16  grf117.txt   52   312   14285
17  grf118.txt   62   434   23988
18  grf119.txt   72   576   37303
19  grf120.txt   82   738   54800
20  grf121.txt   92   920   77049
21  grf122.txt  102  1122  104620
22  grf123.txt   62   372   20735
23  grf124.txt   74   518   34824
24  grf125.txt   86   688   54159
25  grf126.txt   98   882   79568
26  grf127.txt  110  1100  111879
27  grf128.txt  122  1342  151920
28  grf129.txt

In [7]:

# Check how fast do we solve the TSP for each instance and add it to a column called "TSP_time"
def add_tsp_time():
    if "TSP_time" in res.columns:
        return
    res["TSP_time"] = 0.0
    for i, instance in enumerate(instances):
        model = GurobiTSPModel(instance)
        model.formulate()
        model.solve_to_optimality()

        tour = model.get_tour()
        obj = instance.compute_objective(tour)
        instance.save_solution(tour, obj)
        
        res.iloc[i, res.columns.get_loc("TSP_time")] = model.get_model().Runtime

add_tsp_time()
print(res)

         name   N     A        R   TSP_time
0    grf1.txt  20   300     5651   0.349258
1    grf2.txt  20   300     5708   0.353478
2    grf3.txt  20   300    55306   0.293293
3    grf4.txt  20   300    58208   0.633875
4    grf5.txt  20   200     1880   0.031983
5    grf6.txt  20   200     1732   0.065688
6    grf7.txt  40  1200    47777   0.743205
7    grf8.txt  40  1200    47281   2.674696
8    grf9.txt  40  1200   909054   0.830640
9   grf10.txt  40  1200   881876   6.397891
10  grf11.txt  40   800    15490   0.163518
11  grf12.txt  40   800    15650   1.811374
12  grf13.txt  60  2700   162514   1.171736
13  grf14.txt  60  2700   160095  87.285900
14  grf15.txt  60  2700  4527944   4.346083
15  grf16.txt  60  2700  4478927  24.728470
16  grf17.txt  60  1800    53313   0.212033
17  grf18.txt  60  1800    53472  25.621989
18  grf19.txt  20   300     5708   0.221357
19  grf20.txt  40  1200    47281   1.341986
20  grf21.txt  60  2700   160095  94.326014
